In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import textwrap
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [2]:
df = pd.read_csv('RussianOligarchPropertyMap.csv')

In [3]:
df.columns

Index(['Still Own?', 'Sale Date', 'Name', 'Oligarch_Description',
       'Full_Address', 'City', 'Sale Price', 'Property Description',
       'Property Type', 'Story/Reference Link', 'Story/Reference Link #2'],
      dtype='object')

In [4]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [5]:
df['loc'] = df['Full_Address'].apply(geolocator.geocode, timeout=10)

In [6]:
df['loc']

0     (7064 Fisher Island Dr #7064, Miami Beach, FL ...
1     (525 N County Rd, Palm Beach, FL 33480, USA, (...
2     (9 E 75th St, New York, NY 10021, USA, (40.774...
3     (15 E 75th St, New York, NY 10021, USA, (40.77...
4     (215 E 75th St, New York, NY 10021, USA, (40.7...
5     (11 E 64th St, New York, NY 10065, USA, (40.76...
6     (12 Gay St, New York, NY 10014, USA, (40.73360...
7     (785 5th Ave, New York, NY 10022, USA, (40.764...
8     (15 Central Park West, 15 Central Park West, N...
9     (26 E Star Island Dr, Miami Beach, FL 33139, U...
10    (2711 S Ocean Dr PH 1, Hollywood, FL 33019, US...
11    (Trousdale Estates, Beverly Hills, CA 90210, U...
12    (1200 Ridge Wildcat Dr, Snowmass, CO 81615, US...
13    (303 Aspen Way, Snowmass Village, CO 81615, US...
14    (33 E 74th St, New York, NY 10021, USA, (40.77...
Name: loc, dtype: object

In [7]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [8]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,Still Own?,Sale Date,Name,Oligarch_Description,Full_Address,City,Sale Price,Property Description,Property Type,Story/Reference Link,Story/Reference Link #2,loc,point,lat,lon,altitude
0,No,December 2018,Aras Agalarov,Azerbaijani Russian billionaire and real estat...,"7064 Fisher Island Drive, Unit 7064",Miami,$8.46M,Fisher Island Condo,Residential,https://therealdeal.com/miami/2018/12/03/azerb...,NaN,"(7064 Fisher Island Dr #7064, Miami Beach, FL ...","(25.7623516, -80.1473307, 0.0)",25.762352,-80.147331,0.0
1,No,July 2019,Dmitry Rybolovlev,Russian fertilizer billionaire Dmitry Rybolovl...,"525 North County Road, Palm Beach, Florida",Palm Beach,$37.3M,Empty 2-acre lot,Residential,https://therealdeal.com/miami/2019/07/08/russi...,NaN,"(525 N County Rd, Palm Beach, FL 33480, USA, (...","(26.732162, -80.0367618, 0.0)",26.732162,-80.036762,0.0
2,No,September 2018,Roman Abramovich,"Businessman Roman Abramovich, who made his for...","9 East 75th street, Manhattan, New York",NYC,$74M,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(9 E 75th St, New York, NY 10021, USA, (40.774...","(40.77422199999999, -73.96456789999999, 0.0)",40.774222,-73.964568,0.0
3,No,September 2018,Roman Abramovich,"Businessman Roman Abramovich, who made his for...","15 East 75th street, Manhattan, New York",NYC,16.5M,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(15 E 75th St, New York, NY 10021, USA, (40.77...","(40.774133, -73.9644059, 0.0)",40.774133,-73.964406,0.0
4,No,September 2018,Roman Abramovich,"Businessman Roman Abramovich, who made his for...","215 East 75th street, Manhattan, New York",NYC,$900K,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(215 E 75th St, New York, NY 10021, USA, (40.7...","(40.7715223, -73.9583525, 0.0)",40.771522,-73.958353,0.0
5,Now owned by relatives,NaN,Oleg Deripaska,Russian oligarch and billionaire founder of Ru...,"11 East 64th Street, Manhattan, New York",NYC,$42.5M,NaN,Residential,https://nypost.com/2022/02/27/heres-where-russ...,NaN,"(11 E 64th St, New York, NY 10065, USA, (40.76...","(40.7671909, -73.96962649999999, 0.0)",40.767191,-73.969626,0.0
6,Now owned by relatives,NaN,Oleg Deripaska,Russian oligarch and billionaire founder of Ru...,"12 Gay Street, Manhattan, New York",NYC,$4.5M,NaN,Residential,https://nypost.com/2022/02/27/heres-where-russ...,NaN,"(12 Gay St, New York, NY 10014, USA, (40.73360...","(40.7336045, -74.00060979999999, 0.0)",40.733604,-74.000610,0.0
7,Yes,May 2018,Eugene Shvidler,"Russian-born American Eugene Shvidler, a gold ...","785 Fifth Avenue, Apt 17AB, Manhattan, New York",NYC,$24.5M,NaN,Residential,https://nypost.com/2022/02/27/heres-where-russ...,NaN,"(785 5th Ave, New York, NY 10022, USA, (40.764...","(40.76457569999999, -73.97252259999999, 0.0)",40.764576,-73.972523,0.0
8,NaN,NaN,Dmitry Rybolovlev,Russian fertilizer billionaire Dmitry Rybolovl...,"15 Central Park West, Manhattan, New York",NYC,$88M,NaN,Residential,https://nypost.com/2022/02/27/heres-where-russ...,NaN,"(15 Central Park West, 15 Central Park West, N...","(40.7697189, -73.9810746, 0.0)",40.769719,-73.981075,0.0
9,Yes,NaN,Vlad Doronin,Russian-born American billionaire real estate ...,"26 Star Island drive, Miami Beach",Miami,$16M,NaN,Residential,NaN,NaN,"(26 E Star Island Dr, Miami Beach, FL 33139, U...","(25.77965, -80.15058839999999, 0.0)",25.779650,-80.150588,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [9]:
#df.at[3,'building_name']=('87 Park')

In [10]:
df.columns

Index(['Still Own?', 'Sale Date', 'Name', 'Oligarch_Description',
       'Full_Address', 'City', 'Sale Price', 'Property Description',
       'Property Type', 'Story/Reference Link', 'Story/Reference Link #2',
       'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=4)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Russian Oligarch Real Estate Transactions/investments')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    desc = r['Oligarch_Description']
    splitted = textwrap.wrap(desc, 50)
    desc = '<br>'.join(splitted)
    info = (r['Name'],r['Full_Address'],desc)
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Brief Bio: \
    {bold_start}{info[2]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))

m

In [12]:
m.save('index.html')